In [5]:
!pip install streamlit
!pip install pyngrok==4.1.1
from pyngrok import ngrok

# Configuração do authtoken do ngrok
ngrok.set_auth_token('2ftUquXmuNm34pdtD10J1R0DeZL_6rtMraLwodXeGtZJEkg9R')

# Encerra qualquer sessão ngrok anterior
ngrok.kill()

# Roda o Streamlit em segundo plano
get_ipython().system_raw('streamlit run app.py &')

# Estabelece um túnel para o port 8501 (porta padrão do Streamlit)
ngrok_tunnel = ngrok.connect(8501)
print('Seu dashboard está rodando em:', ngrok_tunnel.public_url)


ERROR:pyngrok.process:t=2024-05-02T03:22:45+0000 lvl=eror msg="failed to auth" obj=tunnels.session err="Your ngrok-agent version \"2.3.41\" is too old. The minimum supported agent version for your account is \"3.2.0\". Please update to a newer version with `ngrok update`, by downloading from https://ngrok.com/download, or by updating your SDK version. Paid accounts are currently excluded from minimum agent version requirements. To begin handling traffic immediately without updating your agent, upgrade to a paid plan: https://dashboard.ngrok.com/billing/subscription.\r\n\r\nERR_NGROK_121\r\n"



PyngrokNgrokError: The ngrok process errored on start: Your ngrok-agent version "2.3.41" is too old. The minimum supported agent version for your account is "3.2.0". Please update to a newer version with `ngrok update`, by downloading from https://ngrok.com/download, or by updating your SDK version. Paid accounts are currently excluded from minimum agent version requirements. To begin handling traffic immediately without updating your agent, upgrade to a paid plan: https://dashboard.ngrok.com/billing/subscription.\r\n\r\nERR_NGROK_121\r\n.

In [6]:
# Baixe a versão mais recente do ngrok
!wget -q -c https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px

# Configurações iniciais de página
st.set_page_config(page_title='Dashboard de Vendas', layout='wide', page_icon='📊')

# Opção para escolher o tema
theme_choice = st.sidebar.radio('Escolha o tema da página:', ('Claro', 'Escuro'))

if theme_choice == 'Escuro':
    st.write('<style>body { background-color: #262730; color: white; }</style>', unsafe_allow_html=True)
else:
    st.write('<style>body { background-color: #FFFFFF; color: black; }</style>', unsafe_allow_html=True)

# Lendo e preparando os dados
df = pd.read_csv("supermarket_sales.csv", sep=";", decimal=",")
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date")
df["Month"] = df["Date"].apply(lambda x: str(x.year) + "-" + str(x.month))
df["Day"] = df["Date"].dt.day

# Sidebar - Filtros
month = st.sidebar.selectbox("Mês", df["Month"].unique())
city = st.sidebar.selectbox("Cidade", df["City"].unique(), index=0)

# Filtragem dos dados
df_filtered = df[(df["Month"] == month) & (df["City"] == city)]

# Colunas para os gráficos
col1, col2 = st.columns(2)
col3, col4, col5 = st.columns(3)

# Gráficos
# Faturamento por dia
fig_date = px.bar(df_filtered, x="Date", y="Total", color="City", title="Faturamento por dia")
col1.plotly_chart(fig_date, use_container_width=True)

# Faturamento por tipo de produto
fig_prod = px.bar(df_filtered, x="Product line", y="Total", color="City", title="Faturamento por tipo de produto", orientation="h")
col2.plotly_chart(fig_prod, use_container_width=True)

# Faturamento por filial
city_total = df_filtered.groupby("City")[["Total"]].sum().reset_index()
fig_city = px.bar(city_total, x="City", y="Total", title="Faturamento por filial")
col3.plotly_chart(fig_city, use_container_width=True)

# Faturamento por tipo de pagamento
fig_kind = px.pie(df_filtered, values="Total", names="Payment", title="Faturamento por tipo de pagamento")
col4.plotly_chart(fig_kind, use_container_width=True)

# Avaliação média por filial
fig_rating = px.bar(df_filtered, y="Rating", x="City", title="Avaliação média por filial")
col5.plotly_chart(fig_rating, use_container_width=True)
